### Agents
Agents will provide a way for us to add additional sources of information that will build on our RAG approach.  We may need to execute code or search different information sources to build a complex context to answer user queries.

### Tools

In [3]:
%pip install langchain_community -q
%pip install tavily-python -q
%pip install python-dotenv -q
%pip install langchainhub -q
%pip install wikibase-rest-api-client mediawikiapi -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/8c/74/11971c651e88169ffdba829e106dadadb64480748c3c5ecc08e55d71fc13/boto3-1.38.4-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.39.0,>=1.38.4 from https://files.pythonhosted.org/packages/6b/67/93837ce62b4e49eaf77f076c4781e942386acb70ad1923429df8c6b1371e/botocore-1.38.4-py3-none-any.whl.metadata
  Obtaining dependency information for jmespath<2.0.0,>=0.7.1 from https://files.pythonhosted.org/packages/31/b4/b9b800c45527aadd64d5b442f9b932b00648617eb5d63d2c7a6587b7cafc/jmespath-1.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.13.0,>=0.12.0 from https://files.pythonhosted.org/packages/89/64/d2b49620039b82688aeebd510bd62ff4cdcdb86cbf650cc72ae42c5254a3/s3transfer-0.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3!=2.2.0,<3,>=1.25.4 from https://files.pythonhosted.org/packages/6b/11/cc635220681e93a0183390e26485430c

In [10]:
import os
import boto3

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain_community.chat_models import BedrockChat
from langchain_core.messages import AIMessage, HumanMessage
from langchain import hub
from dotenv import load_dotenv

load_dotenv()
# Create the AWS client for the Bedrock runtime with boto3
aws_client = boto3.client(service_name="bedrock-runtime")

### Agent Setup
We need three components to make up our agent
1) Tools we plan to use to execute respond to input
2) LLM we plan to use as our logic executor
3) Agent type we plan to use for this use case


#### Homework
Find a useful tool other than Wikidata or Tavily to explore what it can do

In [7]:
import langchain
import langchain_community

print("LangChain version:", langchain.__version__)
print("LangChain Community version:", langchain_community.__version__)

LangChain version: 0.3.24
LangChain Community version: 0.3.23


In [8]:
!pip uninstall langchain-community -y
!pip install langchain-community --upgrade

Found existing installation: langchain-community 0.3.23
Uninstalling langchain-community-0.3.23:
  Successfully uninstalled langchain-community-0.3.23
  Using cached langchain_community-0.3.23-py3-none-any.whl.metadata (2.5 kB)
Using cached langchain_community-0.3.23-py3-none-any.whl (2.5 MB)


In [4]:
pip install langchain serperdevtool

ERROR: Could not find a version that satisfies the requirement serperdevtool (from versions: none)
ERROR: No matching distribution found for serperdevtool
Note: you may need to restart the kernel to use updated packages.


In [19]:
# Your tool here
# 1 Start with a simple web search tool:
#tools = -> FIND A GOOD TOOL
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.tools import Tool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

serper_wrapper = GoogleSerperAPIWrapper(api_key='SERPER_API_KEY')

serper_tool = Tool(
    name="Google Serper Search",
    func=serper_wrapper.run,
    description="Use this tool to search the web when you need to find recent or factual information"
)

tools = [serper_tool]

In [20]:
# 2 LLM, Let's use Claude's Haiku
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# Grab our LLM of choice
model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 0.9,
    "stop_sequences": ["\n\nHuman"],
}
llm = BedrockChat(
    client=aws_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

# Get the prompt to use with our agent
# The hub provides sample templates for each agent type
prompt = hub.pull("hwchase17/structured-chat-agent")

# print("Tools:", tools)
# print("Prompt", prompt)



/Users/oscarluo/Documents/GSB570 Generative AI/gsb570env/lib/python3.11/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Using your new tool see if you can get the expected output

In [21]:
# 3 Construct the create_structured_chat_agent agent
# https://python.langchain.com/v0.1/docs/modules/agents/agent_types/
agent = create_structured_chat_agent(llm, tools, prompt)

# run the agent
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

agent_executor.invoke({"input": "What happened with the economy today?"})



> Entering new AgentExecutor chain...
Okay, let me try to find some recent information on the economy today.

Action:
```json
{
  "action": "Google Serper Search",
  "action_input": "economy news today"
}
```
Find the latest economic news, current events and headlines, as well as blogs and video from CNBC.com. Economy · Lunch Break: Trump vs. Amazon · Consumers Give Thumbs Down on Tariffs · US Consumer Confidence Plummets to Covid-Era Low as Trade War Stokes Anxiety. The latest news on the economy, job market, real estate industry and workplace. Includes coverage of the Federal Reserve and government spending at the ... Is the 2025 recession coming? Forecasters are roughly split, with nearly 4 in 10 figuring more than 50% odds of a downturn, according to a ... Read about thriving and emerging markets, the global economy, and the latest financial data and forecasts. Latest news and insight about the impact of current events on our economy from MarketWatch. The latest news on the econo

{'input': 'What happened with the economy today?',
 'output': "- Consumer confidence has dropped to a COVID-era low due to concerns over the trade war and tariffs. \n\n- There are mixed forecasts on whether a recession will hit by 2025, with around 40% of forecasters predicting over 50% odds of a downturn.\n\n- The latest economic data includes a drop in new home sales for March and the Fed's Beige Book release providing an overview of economic conditions.\n\n- News outlets like CNBC, MarketWatch, and Bloomberg are covering the latest developments and impacts on the economy from current events like the trade disputes."}